In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
c2c = pd.read_csv("/Users/valentinlepinay/dev/AFP_Forecast_Alpha_Using_ML/C2C/A.csv",
                  dtype={'PERMNO': np.str, "TICKER": np.str, "date": np.str},
                  usecols=['C2C','indexC2C', 'PERMNO', 'TICKER', 'date'])
c2c = c2c.loc[c2c.date.str[:4] == '2001']
c2c.head()

,PERMNO,date,TICKER,C2C,indexC2C
0,87432,20010102,A,-0.070776,-0.028
1,87432,20010103,A,0.103194,0.050
2,87432,20010104,A,0.037862,-0.011
3,87432,20010105,A,-0.054721,-0.026
4,87432,20010108,A,-0.032917,-0.002


In [3]:
# C2C - rf = alpha + b_mkt * (exret - rf) + b_smb * SMB + b_hml * HML

path = '~/dev/AFP_Forecast_Alpha_Using_ML/FF3/Beta-FamaFrench3F-2001.csv'
df = pd.read_csv(path, dtype={'PERMNO': np.str, "TICKER": np.str, "DATE": np.str},
                usecols=['PERMNO', 'TICKER', 'DATE', 'RET', 'exret',
                        'alpha', 'b_mkt', 'b_smb', 'b_hml']
                )
df = df.loc[df.TICKER == 'A']
df['date'] = pd.to_datetime(df['DATE'], format="%Y/%m/%d").dt.strftime('%Y%m%d')
del df['DATE']
df.head()

,PERMNO,RET,alpha,b_mkt,b_smb,b_hml,exret,TICKER,date
172876,87432,-1.5199%,0.0001,1.3080,-0.1780,-1.5969,1.1326%,A,20011231
172877,87432,0.3814%,-0.0000,1.3053,-0.1849,-1.5966,-0.0308%,A,20011228
172878,87432,2.6335%,-0.0001,1.2976,-0.1836,-1.6077,1.1505%,A,20011227
172879,87432,-1.4381%,-0.0001,1.2939,-0.1896,-1.5914,-2.4350%,A,20011226
172880,87432,2.0036%,0.0001,1.3123,-0.1737,-1.5209,2.1110%,A,20011224


In [4]:
df = df.merge(c2c, on=['PERMNO', 'TICKER', 'date'], how='inner')
del c2c

In [5]:
df['RET'] = pd.to_numeric(df['RET'].str[:-1], errors='coerce', downcast='float') * 1.0e-2
df['exret'] = pd.to_numeric(df['exret'].str[:-1], errors='coerce', downcast='float') * 1.0e-2

In [6]:
df.describe()

,RET,alpha,b_mkt,b_smb,b_hml,exret,C2C,indexC2C
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,-0.001709,0.001367,1.412132,0.336418,-0.938817,-0.001122,-0.001709,-0.000480
std,0.043095,0.000928,0.303763,0.759281,0.716140,0.032334,0.043095,0.013569
min,-0.120000,-0.000800,0.924300,-0.368900,-1.750800,-0.091220,-0.120000,-0.049000
25%,-0.027765,0.000800,1.206575,-0.194975,-1.471225,-0.020939,-0.027765,-0.008000
50%,-0.003684,0.001400,1.358400,-0.096100,-1.238500,-0.003564,-0.003685,-0.001000
75%,0.020417,0.001800,1.614025,0.964825,-0.532175,0.015918,0.020417,0.007000
max,0.144027,0.004000,2.078800,1.872700,0.718600,0.106532,0.144027,0.050000


In [7]:
path_factors = '/Users/valentinlepinay/dev/AFP_Forecast_Alpha_Using_ML/Fama-French Factors - Daily Frequency.csv'

factors = pd.read_csv(path_factors, dtype={"date": np.str})
factors['date'] = pd.to_datetime(factors['date'], format="%Y/%m/%d").dt.strftime('%Y%m%d')
factors = factors.loc[factors.date.str[:4] == '2001']
factors.head()

,date,mktrf,smb,hml,rf,umd
0,20010102,-0.0352,-0.0029,0.0196,0.00026,-0.0126
1,20010103,0.0539,0.0060,-0.0415,0.00026,-0.0727
2,20010104,-0.0130,0.0072,0.0022,0.00026,-0.0413
3,20010105,-0.0298,0.0029,0.0219,0.00026,0.0066
4,20010108,-0.0036,-0.0053,0.0096,0.00026,0.0015


In [8]:
df = df.merge(factors, on=['date'], how='inner')
del factors

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
RET,248.0,-0.001709,0.043095,-0.12000,-0.027765,-0.003684,0.020417,0.144027
alpha,248.0,0.001367,0.000928,-0.00080,0.000800,0.001400,0.001800,0.004000
b_mkt,248.0,1.412132,0.303763,0.92430,1.206575,1.358400,1.614025,2.078800
b_smb,248.0,0.336418,0.759281,-0.36890,-0.194975,-0.096100,0.964825,1.872700
b_hml,248.0,-0.938817,0.716140,-1.75080,-1.471225,-1.238500,-0.532175,0.718600
exret,248.0,-0.001122,0.032334,-0.09122,-0.020939,-0.003564,0.015918,0.106532
C2C,248.0,-0.001709,0.043095,-0.12000,-0.027765,-0.003685,0.020417,0.144027
indexC2C,248.0,-0.000480,0.013569,-0.04900,-0.008000,-0.001000,0.007000,0.050000
mktrf,248.0,-0.000531,0.014057,-0.05030,-0.008625,-0.000400,0.007125,0.053900
smb,248.0,0.000704,0.006176,-0.03320,-0.002950,0.001550,0.004200,0.024700


In [10]:
import statsmodels.api as sm
df['C2C-rf'] = df.C2C.values - df.rf.values
df['Mkt-rf'] = df.mktrf.values - df.rf.values
Y = df['C2C-rf'].values
X = df[['Mkt-rf', 'smb', 'hml']].values
model = sm.OLS(Y,sm.add_constant(X))
results = model.fit()

/Users/valentinlepinay/dev/dev/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [11]:
df['simple_y_hat'] = results.predict()
df['simple_b_mkt'] = results.params[3]
df['simple_b_smb'] = results.params[2]
df['simple_b_hml'] = results.params[1]
df['simple_alpha'] = results.params[0]

In [12]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.484
Method:                 Least Squares   F-statistic:                     78.22
Date:                Mon, 11 Feb 2019   Prob (F-statistic):           1.74e-35
Time:                        22:20:07   Log-Likelihood:                 511.94
No. Observations:                 248   AIC:                            -1016.
Df Residuals:                     244   BIC:                            -1002.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.002      0.116      0.908      -0.004       0.004
x1             1.3132      0.207      6.347      0.000       0.906       1.721
x2            -0.1440      0.352     -0.409      0.683      -0.837       0.549
x3            -1.6045      0.339     -4.729      0.000      -2.273      -0.936
==============================================================================
Omnibus:                       16.029   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.141
Skew:                           0.494   Prob(JB):                     4.23e-05
Kurtosis:                       3.987   Cond. No.                         221.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
window = 30
y_ = df['C2C-rf'].values
x_ = df[['Mkt-rf', 'smb', 'hml']].assign(constant=1).values
df['y_hat'] = np.nan
df['ols_alpha'] = np.nan
df['ols_b_mkt'] = np.nan
df['ols_b_smb'] = np.nan
df['ols_b_hml'] = np.nan

df['ols_t_alpha'] = np.nan
df['ols_t_mkt'] = np.nan
df['ols_t_smb'] = np.nan
df['ols_t_hml'] = np.nan


for n in range(window, len(df)):
    y = y_[(n - window):n]
    X = x_[(n - window):n]
    # betas = Inverse(X'.X).X'.y
    betas = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    df.loc[n, 'y_hat'] = betas.dot(x_[n, :])
    df.loc[n,'ols_b_mkt'] = betas[0]
    df.loc[n,'ols_b_smb'] = betas[1]
    df.loc[n,'ols_b_hml'] = betas[2]
    df.loc[n,'ols_alpha'] = betas[3]
    
    Y_hat = X.dot(betas)
    residual = y-Y_hat
    SSR = residual.T.dot(residual)
    SST = (Y-Y.mean()).T.dot(Y-Y.mean())
    R_sq = 1-SSR/SST
    (n_,k) = X.shape
    VarMatrix = np.linalg.inv(X.T.dot(X))*SSR/(n_-k)
    t = betas.T/(np.sqrt(VarMatrix.diagonal()))

    df.loc[n,'ols_t_mkt'] = t[0]
    df.loc[n,'ols_t_smb'] = t[1]
    df.loc[n,'ols_t_hml'] = t[2]
    df.loc[n,'ols_t_alpha'] = t[3]

In [14]:
df.tail(2).T

,246,247
PERMNO,87432,87432
RET,0.103194,-0.070776
alpha,0.0013,0.0012
b_mkt,2.0638,2.0788
b_smb,1.8534,1.8569
b_hml,0.7027,0.7186
exret,0.009568,-0.006549
TICKER,A,A
date,20010103,20010102
C2C,0.103194,-0.070776


In [15]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

/Users/valentinlepinay/dev/dev/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
model = xgb.XGBRegressor()
Y = df['C2C-rf'].values
X = df[['Mkt-rf', 'smb', 'hml']].values
_ = model.fit(X, Y)
y_hat = model.predict(X)
df['XGB_y_hat'] = y_hat

In [ ]:
window = 30
y_ = df['C2C-rf'].values
X_ = df[['Mkt-rf', 'smb', 'hml']].values

df['XGB_rolling_y_hat'] = np.nan

for n in range(window, len(df)):
    Y = y_[(n - window):n]
    X = x_[(n - window):n]
    Y = Y[~np.isnan(Y)]
    X = X[np.where(~np.isnan(Y)), :][0]
    if len(Y) == 0:
        continue
    _ = model.fit(X, Y)
    if len(x_) <= n:
        continue
    y_hat = model.predict([x_[n]])
    df.loc[n, 'XGB_rolling_y_hat'] = y_hat[0]


In [ ]:
df.set_index('date', inplace=True)

In [ ]:
cols = [
    ['C2C-rf', 'y_hat', 'simple_y_hat', 'XGB_y_hat', 'XGB_rolling_y_hat'],
    ['alpha', 'ols_alpha', 'simple_alpha'],
    ['b_mkt', 'ols_b_mkt', 'simple_b_mkt'],
    ['b_smb', 'ols_b_smb', 'simple_b_smb'],
    ['b_hml', 'ols_b_hml', 'simple_b_hml'],
    ['ols_t_mkt', 'ols_t_smb', 'ols_t_hml', 'ols_t_alpha']
]

for c in cols:
    df[c].plot(figsize=(10, 7))
    plt.show()